در این سند قصد داریم چند به چهار شیوه‌ی متفاوت اطلاعات را بازیابی کنیم و روی آنها جستجو انجام بدهیم.


ابتدا قسمت های مشترک هر چهار شیوه را وارد می‌کنیم:

In [4]:
! pip install fasttext
! pip install sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached huggingface_hub-0.8.1-py3-none-any.whl (101 kB)
  Using cached transformers-4.21.0-py3-none-any.whl (4.7 MB)
  Using cached sentencepiece-0.1.96-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached torchvision-0.13.0-cp38-cp38-manylinux1_x86_64.whl (19.1 MB)
  Using cached torch-1.12.0-cp38-cp38-manylinux1_x86_64.whl (776.3 MB)
  Using cached filelock-3.7.1-py3-none-any.whl (10 kB)
  Using cached typing_extensions-4.3.0-py3-none-any.whl (25 kB)
  Using cached tokenizers-0.12.1-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)


In [4]:
from commons.documents_reader import get_documents
from commons.hazm_tokenizer import HazmTokenizer
from searchers.fasttext_searcher import FasttextSearcher
from searchers.tfidf_searcher import TfidfSearcher
from searchers.transformer_searcher import TransformerSearcher

حال تمام متون پردازشی و تمام متن های جستجویی را می‌خوانیم:

In [5]:
documents = get_documents()

queries = open('queries.txt').readlines()

تابع زیر یک جستجوگر و تعدادی متن جستوجویی به عنوان ورودی می‌گیرد و بعد از انجام جستجو نتیجه‌ی آن را چاپ می‌کند:

In [6]:
def search_queries_by_searcher(test_searcher, test_queries):
    test_searcher.add_documents(documents)
    for query in test_queries:
        print("Query:")
        print(query)

        print("Results:")
        result = test_searcher.search(query)
        for doc in result:
            print(doc.id + ": " + doc.content[:120].replace('\n', ''))

حال پیاده سازی هر چهار تا کلاس را در نظر بگیرید:

In [7]:
import fasttext
from scipy import spatial
from commons.document import Document
import pandas
from hazm import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util


class BooleanSearcher:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.documents_by_tokens = {}

    def add_documents(self, documents):
        for document in documents:
            self.__add_document(document)

    def __add_document(self, document):
        tokens = self.tokenizer.tokenize(document.content)
        for token in tokens:
            if not self.documents_by_tokens.__contains__(token):
                self.documents_by_tokens[token] = set()
            self.documents_by_tokens[token].add(document.id)

    def get_included_documents_set(self, token):
        if self.documents_by_tokens.__contains__(token):
            return self.documents_by_tokens[token]
        else:
            return set()

    def search(self, query_text, result_limit=10):
        query_tokens = self.tokenizer.tokenize(query_text)
        query_tokens_included_sets = [self.get_included_documents_set(qt) for qt in query_tokens]
        result_doc_ids = list(set.intersection(*query_tokens_included_sets)) + \
                         list(set.union(*query_tokens_included_sets))
        result_doc_ids = result_doc_ids[:result_limit]
        return [Document.get_document_by_id(doc_id) for doc_id in result_doc_ids]
    

class FasttextSearcher:
    def __init__(self):
        self.model = fasttext.load_model(
            '/home/mohammad/Desktop/Advanced_Information_Retrieval/hw3/hw3_project/fasttext_model/mmd-model.bin')
        self.vector_by_document = {}

    def add_documents(self, documents):
        for document in documents:
            content = document.content.replace('\n', ' ')
            self.vector_by_document[document.id] = self.model.get_sentence_vector(content)

    def search(self, query_text, result_limit=10):
        query_text = query_text.replace('\n', ' ')
        query_vector = self.model.get_sentence_vector(query_text)
        distances = []
        for doc_id, doc_vector in self.vector_by_document.items():
            distances.append((doc_id, 1 - spatial.distance.cosine(query_vector, doc_vector)))
        distances.sort(key=lambda x: x[1], reverse=True)
        return [Document.get_document_by_id(item[0]) for item in distances[:min(result_limit, len(distances))]]

    

class TfidfSearcher:
    def __init__(self):
        self.tf_idf_vector_maker = TfidfVectorizer(norm='l2', tokenizer=word_tokenize)
        self.data_frame = None
        self.query_tokenize = None
        self.documents = None

    def add_documents(self, documents):
        self.documents = documents
        contents = [doc.content for doc in documents]
        tf_idf = self.tf_idf_vector_maker.fit_transform(contents)
        self.data_frame = pandas.DataFrame(tf_idf.todense(), columns=self.tf_idf_vector_maker.get_feature_names_out())
        self.query_tokenize = self.tf_idf_vector_maker.build_tokenizer()

    def search(self, query_text, result_limit=10):
        query_tokens = self.query_tokenize(query_text)
        query_data_frame = self.data_frame[query_tokens].copy()
        query_data_frame.loc[:, "sum"] = query_data_frame.sum(axis=1).array
        result = query_data_frame.nlargest(result_limit, "sum")
        return [self.documents[i] for i in result.index]


    
class TransformerSearcher:
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.vector_by_document = {}

    def add_documents(self, documents):
        for document in documents:
            self.vector_by_document[document.id] = self.model.encode(document.content)

    def search(self, query_text, result_limit=10):
        query_vector = self.model.encode(query_text)
        distances = []
        for doc_id, doc_vector in self.vector_by_document.items():
            distances.append((doc_id, util.cos_sim(query_vector, doc_vector)))
        distances.sort(key=lambda x: x[1], reverse=True)
        return [Document.get_document_by_id(item[0]) for item in distances[:min(result_limit, len(distances))]]


ابتدا boolean را آزمایش می‌کنیم:

In [10]:
tokenizer = HazmTokenizer()
searcher = BooleanSearcher(tokenizer)
search_queries_by_searcher(searcher, queries[:5])

Query:
بازسازي شهر زلزله زده بم

Results:
HAM2-821014-108.ham: رئيس مجلس در بمبه گزارش خبرنگار پارلماني ايسنا، حجت  الاسلام والمسلمين مهدي كروبي كه به منظور بازديد از مناطق زلزله زد
HAM2-821018-127.ham: در نشست با وزير فرهنگ و ارشاد اسلامي پيشنهاد شد؛برنامه هاي جامعه فرهنگي و هنري براي كمك به مردم مناطق  زلزله زدهبنيا
HAM2-830707-090.ham: گله شهردار بم از روند بازسازي اين شهرايسنا: مردم بم وضع معيشتي مناسبي ندارند و جيره غذايي متعلق به آنان فقط در دو ماهه
HAM2-821108-102.ham: بم در بلاتكليفيشوراي شهر و شهرداري تهران خدمات رساني به بم را گسترش مي دهندگروه شهري: اعضاي شوراي شهر و مسئولان شهرد
HAM2-831005-128.ham: يك سال پس از زلزلهزندگي با اميدمديريت شهري تهران برنامه اردوهاي دانش آموزي بم را استمرار بخشند تا به سهم خود بتوانند
HAM2-821014-070.ham: شمار تلفات زلزله بم حدود 35 هزار نفر برآورد شدجانشين فرماندهي لشكر 41 ثارالله و جانشين قرارگاه عملياتي قدس نيروي زميني
HAM2-830430-112.ham: رئيس مجمع تشخيص مصلحت نظام:بازسازي بم بايد براي آيندگان الگو شودكرمان - خبرگزاري جمهوري اسلامي: رئيس  م

حال نوبت tfidf است:

In [8]:
searcher = TfidfSearcher()
search_queries_by_searcher(searcher, queries[:5])

Query:
بازسازي شهر زلزله زده بم

Results:
HAM2-831002-023.ham: گردهمايي معماران در واشنگتنبم جديد را چگونه بسازيم- afp در آستانه سالگرد زلزله ويرانگر بم، معماران و كارشناسان به ام
HAM2-830506-029.ham: بازسازي شهر بم؛ فرصت ها و تهديدهابا توجه به ضعف و ناكارآيي دانش مربوط به بازسازي در كشور ما و نيز اقداماتي كه تاكنون د
HAM2-831005-128.ham: يك سال پس از زلزلهزندگي با اميدمديريت شهري تهران برنامه اردوهاي دانش آموزي بم را استمرار بخشند تا به سهم خود بتوانند
HAM2-821010-030.ham: بازسازي بمبا آنكه شاد باش گفتن به شهروندان بمي در حال كنون بي معنا به نظر مي رسد، اما به هر حال بازسازي اين شهر مي توا
HAM2-831001-076.ham: در گفت وگوي اختصاصي همشهري با مهندس سعيدي كياجزئيات آخرين برنامه هاي ستاد بازسازي بم تشريح شدرضايت مردم، هويت تاريخي
HAM2-831003-183.ham: مهندس محمد سعيدي كيا مسئول بازسازي بم در گفت وگو با همشهريكارنامه بازسازي بمدرفرودگاه بم شرط گذاشت:«اگر مي خواهيد مص
HAM2-831002-077.ham: مراحل بازسازي ارگ بم تا چهار سال ديگر به پايان مي رسدفارس: مدير ميراث فرهنگي و گردشگري استان كرمان گفت: ك

برای fasttext ابتدا یک مدل مخصوص برای داده‌های خودمان باید train کنیم و با این اجرای ساده آن را انجام دادیم:

In [ ]:
from commons.documents_reader import get_documents

current_path = '/home/mohammad/Desktop/Advanced_Information_Retrieval/hw3/hw3_project/fasttext_model'

all_contents_file_path = current_path + '/all_contents.txt'
with open(all_contents_file_path, 'w') as file:
    for document in documents:
        file.write(document.content)

model = fasttext.train_supervised(all_contents_file_path)
model.save_model(current_path + '/mmd-model.bin')

حال آن را آزمایش می‌کنیم:

In [9]:
searcher = FasttextSearcher()
search_queries_by_searcher(searcher, queries[:5])

Query:
بازسازي شهر زلزله زده بم

Results:
HAM2-821014-059.ham: كلاف كشي  بناي ارگدر پي مجروح شدن افشين ابراهيمي مدير پايگاه پژوهشي ارگ بم كه در زلزله اخير بم اتفاق افتاده و مصوبات س
HAM2-821226-026.ham: معجزه هزار واحديبايد به وزارت مسكن و شهرسازي ماشاءالله بگوييم تا چشم نخورد. قرار است پس از 3 ماه از گذشت زلزله بم، عمل
HAM2-831205-110.ham: نشست مطبوعاتي باشگاه استقلال امروز برگزار مي شودگروه ورزشي: در آستانه دربي پايتخت، نشست مطبوعاتي باشگاه استقلال امروز 
HAM2-850330-106.ham: برنامه بازيهاي امروزساعت 30/17 آلمان _ اكوادورساعت 30/17 لهستان _ كاستاريكاساعت 30/22 انگليس _ سوئدساعت30/22پارا
HAM2-821028-021.ham: احداث شهر بم در محل فعلي آنمعاون اداري، مالي و برنامه ريزي استانداري كرمان گفته است كه شهر بم در محل فعلي آن احداث خوا
HAM2-830730-081.ham: توسط مجمع تشخيص مصلحت نظامسياست هاي بخش گردشگري بررسي مي شودايرنا:معاون  رييس  جمهور و رييس  سازمان  ميراث  فرهنگي  
HAM2-820814-056.ham: بزرگترين  دايرةالمعارف  نهج البلاغه  در نمايشگاه  قرآن  عرضه  شدگروه علمي فرهنگي: بزرگترين  دايرة المعارف

در آخر transformer را آزمایش می‌کنیم:

In [10]:
searcher = TransformerSearcher()
search_queries_by_searcher(searcher, queries[:5])

Query:
بازسازي شهر زلزله زده بم

Results:
HAM2-830120-086.ham: بهارستاناي گل بهار! اي شكوه زعفراناي گل نرگس!اي شبچراغ درخشان بر زر سرخ!اين چنين دردمندم رها مسازابونواس - ترج
HAM2-831217-043.ham: بهارستاناهل دوزخ در دوزخ خوش تر باشند كه اندر دنيا، زيرا از حق با خبر باشند و در دنيا بي خبرند.فيه مافيه - مولوي
HAM2-841207-097.ham: برداشت بعدي...!علي درويش* كسي كه به او «زياد برمي خورد» با ديگران «زياد برخورد مي كند»* براي «پيشرفت» بايد مدام «پ
HAM2-821021-047.ham: حيات وحش ايران4 - يوزپلنگ آسيايي
HAM2-841217-117.ham: نگاهفريد مرتضويورود و خروج پرندگان در استان سمنان به دليل عدم شيوع آنفولانزاي مرغي كنترل خواهد شد
HAM2-841204-105.ham: برداشت بعدي...!-علي درويش* «محتاط باش، محتاج نباش».* راضي نشو، رازي از ديگران برملا كني.* وقتي «گوشي را مي كشي» پي
HAM2-830116-094.ham: بهارستانبذار بارون ماچت كنهبذار بارون مث آبچك نقرهرو سرت چيكه كنهبذار بارون واست لالايي بگهلنگستون هيوز - ترجم
HAM2-850429-097.ham: واگذاري مراكز تجاري برج ميلاد از ماه آيندهمدير عامل شركت يادمان سازه گفت: قرارداد واگذاري مراك